In [31]:
import os
import tarfile
import rasterio
from rasterio.plot import reshape_as_image
from rasterio.mask import mask
from rasterio.features import rasterize
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import mapping
import numpy as np
import cv2
import matplotlib.pyplot as plt

# contest df load
meta = pd.read_csv("metadata.csv")
train_labels = pd.read_csv("train_labels.csv")
submission = pd.read_csv("submission_format.csv")

# tiff and geojson file paths

train_tiff_paths = meta.image.values
test_tiff_paths = train_tiff_paths[[0, 1, 3, 5, 6]]
train_geo_paths = meta.train.values
test_geo_paths = meta.test.values

# example path

%matplotlib inline

#extracting stac from .tar file

stac_tar = tarfile.open("stac.tar", mode = "r")
stac_tar.extract("stac")

#checking function worked- stac is in wd

os.listdir()

['.ipynb_checkpoints',
 'image_parsing.ipynb',
 'metadata.csv',
 'stac',
 'stac.tar',
 'submission_format.csv',
 'training_images',
 'train_labels.csv']

In [291]:
train_geo_paths

array(['stac/guatemala/mixco_1_and_ebenezer/train-mixco_1_and_ebenezer.geojson',
       'stac/guatemala/mixco_3/train-mixco_3.geojson',
       'stac/st_lucia/castries/train-castries.geojson',
       'stac/st_lucia/dennery/train-dennery.geojson',
       'stac/st_lucia/gros_islet/train-gros_islet.geojson',
       'stac/colombia/borde_rural/train-borde_rural.geojson',
       'stac/colombia/borde_soacha/train-borde_soacha.geojson'],
      dtype=object)

In [294]:
verified_train.iloc[4514]

id                                                        7a2b59f4
roof_material                                        healthy_metal
verified                                                      True
geometry         POLYGON ((592620.3177654418 502865.8452676701,...
iteration                                                        5
location                                            Colombia_rural
epsg                                                    EPSG:32618
set                                                          train
Name: 2403, dtype: object

In [295]:
vtrain_render_issues

['7a20a57c',
 '7a1da264',
 '7a1da264',
 '7a1da264',
 '7a1da264',
 '7a1da264',
 '7a1da264']

In [287]:
mixco_1_geometries = gpd.read_file(train_geo_paths[0])
mixco_1geometries = mixco_1_geometries.to_crs(crs = {"init" : "EPSG:32616"})
mixco_1_geometries.head(10)

,id,roof_material,verified,geometry
0,7a314698,concrete_cement,True,"POLYGON ((-90.58355 14.62727, -90.58353 14.627..."
1,7a30b066,concrete_cement,True,"POLYGON ((-90.58346 14.62740, -90.58340 14.627..."
2,7a1ce13a,concrete_cement,True,"POLYGON ((-90.58342 14.62725, -90.58341 14.627..."
3,7a2f9712,concrete_cement,True,"POLYGON ((-90.58293 14.62739, -90.58279 14.627..."
4,7a416e42,concrete_cement,True,"POLYGON ((-90.58279 14.62729, -90.58277 14.627..."
5,7a1cb34a,concrete_cement,True,"POLYGON ((-90.58230 14.62710, -90.58229 14.627..."
6,7a3978d6,concrete_cement,True,"POLYGON ((-90.58242 14.62709, -90.58246 14.627..."
7,7a26e7d4,concrete_cement,True,"POLYGON ((-90.58269 14.62691, -90.58268 14.626..."
8,7a35d23a,concrete_cement,True,"POLYGON ((-90.58235 14.62671, -90.58234 14.626..."
9,7a3eef3c,concrete_cement,True,"POLYGON ((-90.58296 14.62675, -90.58303 14.626..."


In [288]:
np.random.seed(1017)
rand_int = np.random.randint(0, len(mixco_1_geometries))
coord = mapping(mixco_1_geometries)["features"][rand_int]["geometry"]
coord

{'type': 'Polygon',
 'coordinates': (((-90.58472852321368, 14.621859511413085),
   (-90.58473399099616, 14.621862244695354),
   (-90.5847263982057, 14.621886255305391),
   (-90.58467054080991, 14.621882999576226),
   (-90.58467539483746, 14.62185806837531),
   (-90.58472852321368, 14.621859511413085)),)}

In [290]:
tiff_paths

array(['stac/guatemala/mixco_1_and_ebenezer/mixco_1_and_ebenezer_ortho-cog.tif',
       'stac/guatemala/mixco_3/mixco_3_ortho-cog.tif',
       'stac/st_lucia/castries/castries_ortho-cog.tif',
       'stac/st_lucia/dennery/dennery_ortho-cog.tif',
       'stac/st_lucia/gros_islet/gros_islet_ortho-cog.tif',
       'stac/colombia/borde_rural/borde_rural_ortho-cog.tif',
       'stac/colombia/borde_soacha/borde_soacha_ortho-cog.tif'],
      dtype=object)

In [289]:
PADDING = 1
mixco_1_tiff = rasterio.open(tiff_paths[0])

masked_image, out_transform = mask(mixco_1_tiff, [coord], 
                                   crop = True, 
                                   pad = True,
                                   filled = False, 
                                   pad_width = PADDING)
roof_image = reshape_as_image(masked_image)
roof_image = cv2.cvtColor(roof_image, cv2.COLOR_RGB2BGR)

print("Image roof type:", mixco_1_geometries.iloc[rand_int].roof_material)
plt.imshow(roof_image)

ValueError: Input shapes do not overlap raster.

In [244]:
train_epsg = ["EPSG:32616", "EPSG:32616", "EPSG:32620", "EPSG:32620", "EPSG:32620", "EPSG:32618", "EPSG:32618"]
train_places = ["Guat_mixco_1", "Guat_mixco_3", "St_Lucia_castries", "St_Lucia_dennery", "St_Lucia_gros",
                "Colombia_rural", "Colombia_soacha"]

test_epsg = ["EPSG:32616", "EPSG:32616", "EPSG:32620", "EPSG:32618", "EPSG:32618"]
test_places = ["Guat_mixco_1", "Guat_mixco_3", "St_Lucia_dennery", "Colombia_rural", "Colombia_soacha"]

def geometry_parse(epsg_list, paths, places, set):
    
    set_geometries = pd.DataFrame()
    
    for num, epsg in enumerate(epsg_list):
        print("Starting loop for {} {}, iteration {}".format(set, epsg, num + 1))
        roof_geometries = gpd.read_file(paths[num])
        roof_geometries = roof_geometries.to_crs(crs = {"init" : epsg})
        roof_geometries["iteration"] = num
        roof_geometries["location"] = places[num]
        roof_geometries["epsg"] = epsg
        roof_geometries["set"] = str(set)
        set_geometries = pd.concat([roof_geometries, set_geometries], axis = 0)
        
    return(set_geometries)

train_geometries = geometry_parse(epsg_list = train_epsg, 
                                  paths = train_geo_paths, places = train_places, set = "train")
test_geometries = geometry_parse(epsg_list = test_epsg, 
                                 paths = train_geo_paths, places = test_places, set = "test")
all_geometries = pd.concat([train_geometries, test_geometries], axis = 0)
all_geometries.head(10)

Starting loop for train EPSG:32616, iteration 1
Starting loop for train EPSG:32616, iteration 2
Starting loop for train EPSG:32620, iteration 3
Starting loop for train EPSG:32620, iteration 4
Starting loop for train EPSG:32620, iteration 5
Starting loop for train EPSG:32618, iteration 6
Starting loop for train EPSG:32618, iteration 7
Starting loop for test EPSG:32616, iteration 1
Starting loop for test EPSG:32616, iteration 2
Starting loop for test EPSG:32620, iteration 3
Starting loop for test EPSG:32618, iteration 4
Starting loop for test EPSG:32618, iteration 5


,id,roof_material,verified,geometry,iteration,location,epsg,set
0,7a407f5a,healthy_metal,True,"POLYGON ((591990.189 505536.478, 592002.866 50...",6,Colombia_soacha,EPSG:32618,train
1,7a29f6b8,healthy_metal,True,"POLYGON ((591981.945 505525.417, 591984.309 50...",6,Colombia_soacha,EPSG:32618,train
2,7a3f90f4,healthy_metal,True,"POLYGON ((591977.459 505539.295, 591984.056 50...",6,Colombia_soacha,EPSG:32618,train
3,7a3fe086,healthy_metal,True,"POLYGON ((592004.692 505515.192, 592017.573 50...",6,Colombia_soacha,EPSG:32618,train
4,7a43babc,healthy_metal,True,"POLYGON ((591986.954 505518.009, 591992.722 50...",6,Colombia_soacha,EPSG:32618,train
5,7a386716,irregular_metal,True,"POLYGON ((591986.119 505511.540, 591998.393 50...",6,Colombia_soacha,EPSG:32618,train
6,7a30c2ea,healthy_metal,True,"POLYGON ((591975.059 505526.461, 591988.202 50...",6,Colombia_soacha,EPSG:32618,train
7,7a277988,healthy_metal,True,"POLYGON ((591988.415 505523.852, 592000.276 50...",6,Colombia_soacha,EPSG:32618,train
8,7a1e7d42,healthy_metal,True,"POLYGON ((591971.928 505502.149, 591984.143 50...",6,Colombia_soacha,EPSG:32618,train
9,7a22b2fe,healthy_metal,True,"POLYGON ((591984.345 505499.853, 591996.648 50...",6,Colombia_soacha,EPSG:32618,train


In [166]:
#del train_geometries, test_geometries, all_geometries

In [246]:
# values match numbers given by competition page

pd.DataFrame({
    "train_roof_n": train_geometries["roof_material"].value_counts(),
    "test_roof_n": test_geometries["roof_material"].value_counts()
})

,train_roof_n,test_roof_n
concrete_cement,1518,1021
healthy_metal,14817,9273
incomplete,669,9
irregular_metal,5241,1609
other,308,285


In [205]:
verified_train.head()

,id,roof_material,verified,geometry,iteration,location,epsg,set
3350,7a1da264,healthy_metal,True,"POLYGON ((113953.990 1619599.897, 113955.798 1...",0,Guat_mixco_1,EPSG:32616,train
1119,7a1d616e,irregular_metal,True,"POLYGON ((113510.706 1619831.498, 113519.086 1...",0,Guat_mixco_1,EPSG:32616,train
1118,7a372a04,irregular_metal,True,"POLYGON ((113508.948 1619828.819, 113518.377 1...",0,Guat_mixco_1,EPSG:32616,train
1117,7a2ba1ac,irregular_metal,True,"POLYGON ((113512.428 1619823.017, 113519.974 1...",0,Guat_mixco_1,EPSG:32616,train
1116,7a40fc28,irregular_metal,True,"POLYGON ((113521.212 1619826.418, 113527.283 1...",0,Guat_mixco_1,EPSG:32616,train


In [242]:
# separating verified/unverified images
# adding new directory for all images

verified_train = train_geometries[train_geometries.verified == True]
unverified_train = train_geometries[train_geometries.verified == False]

os.makedirs("C:/Users/Mark/OpenAI_Carribean/all_carribean_images", exist_ok = True)

verified_train_path = "C:/Users/Mark/OpenAI_Carribean/all_carribean_images/verified_training"
unverified_train_path = "C:/Users/Mark/OpenAI_Carribean/all_carribean_images/unverified_training"
test_path = "C:/Users/Mark/OpenAI_Carribean/all_carribean_images/testing"

os.makedirs(verified_train_path, exist_ok = True)
os.makedirs(unverified_train_path, exist_ok = True)
os.makedirs(test_path, exist_ok = True)

#os.listdir()

os.listdir("all_carribean_images")

['testing', 'unverified_training', 'verified_training']

In [284]:
train_tiff_paths

array(['stac/guatemala/mixco_1_and_ebenezer/mixco_1_and_ebenezer_ortho-cog.tif',
       'stac/guatemala/mixco_3/mixco_3_ortho-cog.tif',
       'stac/st_lucia/castries/castries_ortho-cog.tif',
       'stac/st_lucia/dennery/dennery_ortho-cog.tif',
       'stac/st_lucia/gros_islet/gros_islet_ortho-cog.tif',
       'stac/colombia/borde_rural/borde_rural_ortho-cog.tif',
       'stac/colombia/borde_soacha/borde_soacha_ortho-cog.tif'],
      dtype=object)

In [286]:
verified_train.sort_values(by = ["iteration"], inplace = True)
unverified_train.sort_values(by = ["iteration"], inplace = True)
test_geometries.sort_values(by = ["iteration"], inplace = True)

def geometry_to_image(set, tiff_path, img_folder):
    
    render_issues = []
    image_dims = pd.DataFrame()
       
    for i in range(len(tiff_path)):
        tiff = rasterio.open(tiff_path[i], "r")
        
        try:
            for idx, row in set.iterrows():
                coord = mapping(set)["features"][idx]["geometry"]
                masked_image, out_transform = mask(tiff, [coord],
                                                   crop = True, 
                                                   pad = True,
                                                   filled = False, 
                                                   pad_width = PADDING)
                image = reshape_as_image(masked_image)
                img_path = os.path.join(img_folder, str(row.id + ".png"))
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                cv2.imwrite(img_path, image)

                dimensions = pd.DataFrame({
                    "id": row.id,
                    "width": image.shape[0],
                    "height": image.shape[1],
                    "channel": image.shape[2],
                }, index = [0])

                image_dims = pd.concat([dimensions, image_dims], axis = 0)
                
        except Exception as e:
            #print(e)
            render_issues.append(row.id)
            pass
            
    return(image_dims, render_issues)

AttributeError: 'NoneType' object has no attribute 'reset_index'

In [264]:
verified_train.sort_values(by = ["iteration"], inplace = True)
unverified_train.sort_values(by = ["iteration"], inplace = True)
test_geometries.sort_values(by = ["iteration"], inplace = True)

def geometry_to_image(set, tiff_path, img_folder):
    
    image_dims = pd.DataFrame()
    render_issues = []
    
    try:
        for idx, row in set.iterrows():
            tiff = rasterio.open(tiff_path[i], "r")
            coord = mapping(set)["features"][idx]["geometry"]
            masked_image, out_transform = mask(tiff, [coord],
                                               crop = True, 
                                               pad = True,
                                               filled = False, 
                                               pad_width = PADDING)
            image = reshape_as_image(masked_image)
            img_path = os.path.join(img_folder, str(row.id + ".png"))
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(img_path, image)

            dimensions = pd.DataFrame({
                "id": row.id,
                "width": image.shape[0],
                "height": image.shape[1],
                "channel": image.shape[2],
            }, index = [0])

            image_dims = pd.concat([dimensions, image_dims], axis = 0)
            
    except Exception as e:
        #print(e)
        render_issues.append(idx)
        
    return(image_dims, errors)

In [ ]:
verified_train_dims = geometry_to_image(set = verified_train, 
                                        tiff_path = train_tiff_paths,
                                        img_folder = verified_train_path)

unverified_train_dims = geometry_to_image(set = unverified_train, 
                                          tiff_path = train_tiff_paths,
                                          img_folder = unverified_train_path)

test_dims = geometry_to_image(set = test_geometries, 
                              tiff_path = test_tiff_paths,
                              img_folder = test_path)

In [253]:
#4514 failed to render then stops
verified_train_dims, vtrain_render_issues = geometry_to_image(set = verified_train, 
                                                              tiff_path = train_tiff_paths,
                                                              img_folder = verified_train_path)

In [228]:
#4514 failed to render then stops
unverified_train_dims, unvtrain_render_issues = geometry_to_image(set = unverified_train, 
                                                                  tiff_path = train_tiff_paths,
                                                                  img_folder = unverified_train_path)

In [263]:
verified_train.roof_material.value_counts()

healthy_metal      7381
irregular_metal    5241
concrete_cement    1387
incomplete          668
other               193
Name: roof_material, dtype: int64

In [280]:
unverified_train_dims

{'errors': ['7a254bae'], 'dim_df': Empty DataFrame
 Columns: []
 Index: []}

In [231]:
unverified_train.shape

(7683, 8)

In [266]:
test_dims, test_render_issues = geometry_to_image(set = test_geometries, 
                                                  tiff_path = test_tiff_paths,
                                                  img_folder = test_path)

In [268]:
test_render_issues

['7a28e412', '7a1da264', '7a1da264', '7a1da264', '7a1da264']

In [281]:
test_render_issues
test_geometries[test_geometries.id ==  "7a28e412"]

,id,roof_material,verified,geometry,iteration,location,epsg,set
3988,7a28e412,healthy_metal,False,"POLYGON ((716464.626 1549593.239, 716468.789 1...",2,St_Lucia_dennery,EPSG:32620,test


['7a28e412', '7a1da264', '7a1da264', '7a1da264', '7a1da264']

In [270]:
error_look = verified_train[verified_train.id == "7a1da264"]
error_look.head()

,id,roof_material,verified,geometry,iteration,location,epsg,set
3350,7a1da264,healthy_metal,True,"POLYGON ((113953.990 1619599.897, 113955.798 1...",0,Guat_mixco_1,EPSG:32616,train


In [282]:
vtrain_render_issues

['7a20a57c',
 '7a1da264',
 '7a1da264',
 '7a1da264',
 '7a1da264',
 '7a1da264',
 '7a1da264']

In [241]:
# to delete folder when needed
import shutil
shutil.rmtree(verified_train_path)